## 載入模組

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

## 載入資料

In [2]:
df = pd.read_csv("retail_data_09282020.csv",encoding="utf-8")
df.head()

,CustomerID,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,ProductName,Quantity,UnitPrice,Subtotal,CategoryName,NewInvoiceDate,regionName,regionID
0,16446,581483,2011-12-09,09:15:00,23843,中興米外銷日本之米(圓ㄧ)3Kg,285,169,48165,米油沖泡_米．食用油_食用米_米,2019-12-07,中壢區,327
1,12346,541431,2011-01-18,10:01:00,23166,樂米穀場花東自然農耕特哉鮮白米(圓二)1.5K,272,189,51408,米油沖泡_米．食用油_食用米_米,2019-01-16,蘆竹區,325
2,12901,573008,2011-10-27,12:26:00,84077,一匙靈Attack抗菌EX洗衣精補充-1.5Kg,69,79,5451,日用百貨_家庭清潔_洗衣用品_洗衣精,2019-10-25,大溪區,325
3,13135,554868,2011-05-27,10:52:00,22197,鴻津太國捲(蛋奶素)-370g,66,88,5808,飲料零食_精選餅乾_鹹餅乾_米果,2019-05-25,觀音區,330
4,18087,544612,2011-02-22,10:43:00,22053,IQ Dog 成犬乾狗糧-羊肉13.5kg,62,409,25358,生活休閒_寵物用品_狗_狗乾糧．罐頭,2019-02-20,龍潭區,338


## 資料前處理

In [3]:
def seasonClass(x):
    if int(x.split('-')[1]) > 2 and int(x.split('-')[1]) < 6: #3,4,5月
        return 'spring'
    elif int(x.split('-')[1]) > 5 and int(x.split('-')[1]) < 9: #6,7,8月
        return 'summer'
    elif int(x.split('-')[1]) > 8 and int(x.split('-')[1]) < 12: #9,10,11月
        return 'autumn'
    else: #12,1,2月
        return 'winter'
# 新增季節欄位，根據月份來劃分
df['season'] = df['NewInvoiceDate'].apply(seasonClass)
df.head()

,CustomerID,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,ProductName,Quantity,UnitPrice,Subtotal,CategoryName,NewInvoiceDate,regionName,regionID,season
0,16446,581483,2011-12-09,09:15:00,23843,中興米外銷日本之米(圓ㄧ)3Kg,285,169,48165,米油沖泡_米．食用油_食用米_米,2019-12-07,中壢區,327,winter
1,12346,541431,2011-01-18,10:01:00,23166,樂米穀場花東自然農耕特哉鮮白米(圓二)1.5K,272,189,51408,米油沖泡_米．食用油_食用米_米,2019-01-16,蘆竹區,325,winter
2,12901,573008,2011-10-27,12:26:00,84077,一匙靈Attack抗菌EX洗衣精補充-1.5Kg,69,79,5451,日用百貨_家庭清潔_洗衣用品_洗衣精,2019-10-25,大溪區,325,autumn
3,13135,554868,2011-05-27,10:52:00,22197,鴻津太國捲(蛋奶素)-370g,66,88,5808,飲料零食_精選餅乾_鹹餅乾_米果,2019-05-25,觀音區,330,spring
4,18087,544612,2011-02-22,10:43:00,22053,IQ Dog 成犬乾狗糧-羊肉13.5kg,62,409,25358,生活休閒_寵物用品_狗_狗乾糧．罐頭,2019-02-20,龍潭區,338,winter


In [4]:
df_season = df.groupby('season').agg({'Quantity': lambda x: x.sum()})
df_season

,Quantity
season,
autumn,461841
spring,238769
summer,265445
winter,260575


In [5]:
df_spring = df[df['season'] == 'spring']
df_summer = df[df['season'] == 'summer']
df_autumn = df[df['season'] == 'autumn']
df_winter = df[df['season'] == 'winter']
df_winter

,CustomerID,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,ProductName,Quantity,UnitPrice,Subtotal,CategoryName,NewInvoiceDate,regionName,regionID,season
0,16446,581483,2011-12-09,09:15:00,23843,中興米外銷日本之米(圓ㄧ)3Kg,285,169,48165,米油沖泡_米．食用油_食用米_米,2019-12-07,中壢區,327,winter
1,12346,541431,2011-01-18,10:01:00,23166,樂米穀場花東自然農耕特哉鮮白米(圓二)1.5K,272,189,51408,米油沖泡_米．食用油_食用米_米,2019-01-16,蘆竹區,325,winter
4,18087,544612,2011-02-22,10:43:00,22053,IQ Dog 成犬乾狗糧-羊肉13.5kg,62,409,25358,生活休閒_寵物用品_狗_狗乾糧．罐頭,2019-02-20,龍潭區,338,winter
6,15749,540815,2011-01-11,12:55:00,21108,泡舒洗潔精補充包-檸檬-800g,56,59,3304,日用百貨_家庭清潔_廚房清潔_洗碗精,2019-01-09,觀音區,326,winter
12,16333,543057,2011-02-03,10:50:00,84077,一匙靈Attack抗菌EX洗衣精補充-1.5Kg,51,79,4029,日用百貨_家庭清潔_洗衣用品_洗衣精,2019-02-01,龜山區,336,winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527798,12748,581580,2011-12-09,12:20:00,22721,【寵物用品】富士碗-中,1,69,69,生活休閒_寵物用品_狗_餵食器,2019-12-07,楊梅區,336,winter
527799,12748,581580,2011-12-09,12:20:00,21351,HERAN禾聯 65吋 UHD顯示器 HD-65WDF41,1,35700,35700,大小家電_電視．冰箱．洗衣機_電視_UHD液晶電視,2019-12-07,楊梅區,336,winter
527800,12748,581580,2011-12-09,12:20:00,22698,朵蔓頭皮淨化噴霧,1,399,399,美妝護理_個人清潔_洗髮保養_護髮乳,2019-12-07,楊梅區,336,winter
527801,12748,581580,2011-12-09,12:20:00,22567,雀巢金牌咖啡罐裝深焙風味120g,1,253,253,米油沖泡_咖啡．茶包_咖啡_即溶咖啡,2019-12-07,楊梅區,336,winter


In [6]:
df_spring_buy_rate = df_spring.groupby('StockCode').agg({'Quantity': lambda x: (x.sum())/df_season.loc['autumn']['Quantity']})
df_summer_buy_rate = df_summer.groupby('StockCode').agg({'Quantity': lambda x: (x.sum())/df_season.loc['autumn']['Quantity']})
df_autumn_buy_rate = df_autumn.groupby('StockCode').agg({'Quantity': lambda x: (x.sum())/df_season.loc['autumn']['Quantity']})
df_winter_buy_rate = df_winter.groupby('StockCode').agg({'Quantity': lambda x: (x.sum())/df_season.loc['autumn']['Quantity']})

In [7]:
df_spr = pd.merge(df_spring,df_spring_buy_rate,on='StockCode')
df_sum = pd.merge(df_summer,df_summer_buy_rate,on='StockCode')
df_aut = pd.merge(df_autumn,df_autumn_buy_rate,on='StockCode')
df_win = pd.merge(df_winter,df_winter_buy_rate,on='StockCode')

In [8]:
df1 = pd.concat([df_spr,df_sum,df_aut,df_win])
df1.rename(columns={'Quantity_x': 'Quantity', 'Quantity_y': 'Buy_index'}, inplace=True)

In [9]:
df1.head()

,CustomerID,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,ProductName,Quantity,UnitPrice,Subtotal,CategoryName,NewInvoiceDate,regionName,regionID,season,Buy_index
0,13135,554868,2011-05-27,10:52:00,22197,鴻津太國捲(蛋奶素)-370g,66,88,5808,飲料零食_精選餅乾_鹹餅乾_米果,2019-05-25,觀音區,330,spring,0.002312
1,477,548203,2011-03-29,16:40:00,22197,鴻津太國捲(蛋奶素)-370g,25,88,2200,飲料零食_精選餅乾_鹹餅乾_米果,2019-03-27,龜山區,324,spring,0.002312
2,12931,550122,2011-04-14,12:39:00,22197,鴻津太國捲(蛋奶素)-370g,24,88,2112,飲料零食_精選餅乾_鹹餅乾_米果,2019-04-12,大溪區,330,spring,0.002312
3,17949,546319,2011-03-11,09:23:00,22197,鴻津太國捲(蛋奶素)-370g,22,88,1936,飲料零食_精選餅乾_鹹餅乾_米果,2019-03-09,楊梅區,325,spring,0.002312
4,12931,553928,2011-05-20,10:02:00,22197,鴻津太國捲(蛋奶素)-370g,20,88,1760,飲料零食_精選餅乾_鹹餅乾_米果,2019-05-18,大溪區,330,spring,0.002312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118954,12748,581449,2011-12-08,17:37:00,23462,崇德發水蜜桃醋,1,199,199,飲料零食_果汁．飲料．養生_濃縮飲料．水果醋_濃縮醋,2019-12-06,楊梅區,336,winter,0.000002
118955,1425,581492,2011-12-09,10:03:00,23635,小不叮驅蚊噴霧 120ml-升級版 嬰幼兒用,1,340,340,母嬰保健_育嬰用品_嬰幼兒用品_防護用品,2019-12-07,大園區,337,winter,0.000002
118956,1425,581492,2011-12-09,10:03:00,23644,百研/柔軟海綿菜瓜布2入,1,69,69,日用百貨_家庭清潔_清潔工具_衛浴清潔工具,2019-12-07,大園區,337,winter,0.000002
118957,14446,581538,2011-12-09,11:34:00,23033,PA-112B美式四芯雙接頭5M,1,79,79,3C_手機．通訊．週邊_電話．傳真機_電話配件,2019-12-07,新屋區,327,winter,0.000002


In [10]:
quantiles = df1.quantile(q=[0.2,0.4,0.6,0.8])
quantiles

,CustomerID,InvoiceNo,Quantity,UnitPrice,Subtotal,regionID,Buy_index
0.2,1277.0,545545.0,1.0,69.0,108.0,325.0,0.000158
0.4,13571.0,555924.0,1.0,109.0,190.0,328.0,0.000338
0.6,15007.4,565461.0,2.0,166.0,329.0,333.0,0.000608
0.8,16729.0,574040.0,3.0,279.0,650.0,336.0,0.001182


In [11]:
quantiles = quantiles.to_dict()
quantiles

{'CustomerID': {0.2: 1277.0,
  0.4: 13571.0,
  0.6: 15007.400000000023,
  0.8: 16729.0},
 'InvoiceNo': {0.2: 545545.0, 0.4: 555924.0, 0.6: 565461.0, 0.8: 574040.0},
 'Quantity': {0.2: 1.0, 0.4: 1.0, 0.6: 2.0, 0.8: 3.0},
 'UnitPrice': {0.2: 69.0, 0.4: 109.0, 0.6: 166.0, 0.8: 279.0},
 'Subtotal': {0.2: 108.0, 0.4: 190.0, 0.6: 329.0, 0.8: 650.0},
 'regionID': {0.2: 325.0, 0.4: 328.0, 0.6: 333.0, 0.8: 336.0},
 'Buy_index': {0.2: 0.00015806305633324022,
  0.4: 0.0003377785861367873,
  0.6: 0.0006084345045156233,
  0.8: 0.0011822250514787556}}

In [12]:
def quantity_class(x,k,d):
    if x < 2:
        return 2
    elif x < 3:
        return 3
    elif x < 4:
        return 4
    else :
        return 5
def price_class(x,k,d):
    if x < d[k][0.2]:
        return 1
    elif x < d[k][0.4]:
        return 2
    elif x < d[k][0.6]:
        return 3
    elif x < d[k][0.8]:
        return 4
    else:
        return 5
def buy_index_class(x,k,d):
    if x < d[k][0.2]:
        return 1
    elif x < d[k][0.4]:
        return 2
    elif x < d[k][0.6]:
        return 3
    elif x < d[k][0.8]:
        return 4
    else:
        return 5
df1['quantity_Quartile'] = df1['Quantity'].apply(quantity_class, args=('Quantity',quantiles,))
df1['price_Quartile'] = df1['UnitPrice'].apply(price_class, args=('UnitPrice',quantiles,))
df1['buy_index_Quartile'] = df1['Buy_index'].apply(buy_index_class, args=('Buy_index',quantiles,))
df1 = df1.drop_duplicates(subset=['CustomerID','StockCode','ProductName']).sort_values(by=['CustomerID']).reset_index(drop=True)
df1['rating'] = df1['quantity_Quartile']*0.1 + df1['price_Quartile']*0.4 + df1['buy_index_Quartile']*0.5

df1.head()

,CustomerID,InvoiceNo,InvoiceDate,InvoiceTime,StockCode,ProductName,Quantity,UnitPrice,Subtotal,CategoryName,NewInvoiceDate,regionName,regionID,season,Buy_index,quantity_Quartile,price_Quartile,buy_index_Quartile,rating
0,1,536544,2010-12-01,14:32:00,22664,龍口埔里水粉400g,2,44,88,米油沖泡_泡麵．麵條_米粉．麵條_粉絲,2018-11-29,桃園區,326,autumn,0.000169,3,1,2,1.7
1,1,536544,2010-12-01,14:32:00,22294,康乃馨夜用特長超薄蝶型(14片x4包),2,176,352,美妝護理_女性衛生_衛生棉_夜用,2018-11-29,桃園區,326,autumn,0.000747,3,4,4,3.9
2,1,536544,2010-12-01,14:32:00,21385,家樂福哥斯大黎加濾泡式咖啡粉,1,99,99,米油沖泡_咖啡．茶包_咖啡_咖啡粉/咖啡豆,2018-11-29,桃園區,326,autumn,0.000372,2,2,3,2.5
3,1,536544,2010-12-01,14:32:00,21107,木柄菜匙,1,35,35,生活休閒_日用百貨_餐具/鍋具/廚房配件_砧板、鏟勺,2018-11-29,桃園區,326,autumn,0.000214,2,1,2,1.6
4,1,536544,2010-12-01,14:32:00,22379,美琪天然T3抗菌洗手乳,1,101,101,美妝護理_個人清潔_沐浴用品_進口香皂．抗菌香皂．潔手乳,2018-11-29,桃園區,326,autumn,0.001009,2,2,4,3.0


In [13]:
# 建立商品ID對應商品名稱的Dataframe
df2 = df1.drop(columns=['InvoiceNo','InvoiceDate','InvoiceTime','Quantity','UnitPrice','Subtotal','CategoryName','NewInvoiceDate','regionName','regionID','season','Buy_index'])
product_name = df2.drop_duplicates(subset=['StockCode','ProductName']).drop(columns=['CustomerID','quantity_Quartile','price_Quartile','buy_index_Quartile','rating']).reset_index(drop=True)
product_name.head()

In [15]:
#對個人所有評分做平均
Mean = df2.groupby(by="CustomerID",as_index=False)['rating'].mean()
Mean.head()

,CustomerID,rating
0,1,2.753257
1,2,2.800000
2,3,2.000000
3,4,1.300000
4,5,2.705433


In [16]:
#將平均評分合併，rating欄位名稱一樣，pandas自動變成rating_x, rating_y
Rating_avg = pd.merge(df2,Mean,on='CustomerID')
Rating_avg.head()

,CustomerID,StockCode,ProductName,quantity_Quartile,price_Quartile,buy_index_Quartile,rating_x,rating_y
0,1,22664,龍口埔里水粉400g,3,1,2,1.7,2.753257
1,1,22294,康乃馨夜用特長超薄蝶型(14片x4包),3,4,4,3.9,2.753257
2,1,21385,家樂福哥斯大黎加濾泡式咖啡粉,2,2,3,2.5,2.753257
3,1,21107,木柄菜匙,2,1,2,1.6,2.753257
4,1,22379,美琪天然T3抗菌洗手乳,2,2,4,3.0,2.753257


In [17]:
#normalized rating, 因為每個人的評判'嚴格度'不一樣', ex: '阿嶽真的很嚴格', '他什麼都說好吃'
#新增欄位，值為個別評分 - 平均評分
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head()

,CustomerID,StockCode,ProductName,quantity_Quartile,price_Quartile,buy_index_Quartile,rating_x,rating_y,adg_rating
0,1,22664,龍口埔里水粉400g,3,1,2,1.7,2.753257,-1.053257
1,1,22294,康乃馨夜用特長超薄蝶型(14片x4包),3,4,4,3.9,2.753257,1.146743
2,1,21385,家樂福哥斯大黎加濾泡式咖啡粉,2,2,3,2.5,2.753257,-0.253257
3,1,21107,木柄菜匙,2,1,2,1.6,2.753257,-1.153257
4,1,22379,美琪天然T3抗菌洗手乳,2,2,4,3.0,2.753257,0.246743


In [19]:
#做出userId-itemID的pivot table, 因為每個人可能只買過幾樣商品, 所以是一個很大的稀疏矩陣
final=pd.pivot_table(Rating_avg,values='adg_rating',index='CustomerID',columns='StockCode')
final

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,DCGS0069,DCGS0070,DCGS0076,DCGSSBOY,DCGSSGIRL,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50
CustomerID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.146743,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.494567,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#填空值
#方法一：以userId列的標準化評分的平均填入
#方法二：以商品Id欄的標準化評分的平均填入 <---我們選這個
final_item = final.fillna(final.mean(axis=0))
final_item

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,DCGS0069,DCGS0070,DCGS0076,DCGSSBOY,DCGSSGIRL,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50
CustomerID,,,,,,,,,,,,,,,,,,,,,
1,-0.858445,-0.942859,-0.489305,-0.12973,-0.18104,-1.80607,-0.967031,0.026938,0.636627,0.146743,...,-0.366102,0.036765,0.082684,0.025156,-1.180128,-1.161536,-0.418427,-1.588743,0.015588,-1.518989
2,-0.858445,-0.942859,-0.489305,-0.12973,-0.18104,-1.80607,-0.967031,0.026938,0.636627,0.193614,...,-0.366102,0.036765,0.082684,0.025156,-1.180128,-1.161536,-0.418427,-1.588743,0.015588,-1.518989
3,-0.858445,-0.942859,-0.489305,-0.12973,-0.18104,-1.80607,-0.967031,0.026938,0.636627,0.193614,...,-0.366102,0.036765,0.082684,0.025156,-1.180128,-1.161536,-0.418427,-1.588743,0.015588,-1.518989
4,-0.858445,-0.942859,-0.489305,-0.12973,-0.18104,-1.80607,-0.967031,0.026938,0.636627,0.193614,...,-0.366102,0.036765,0.082684,0.025156,-1.180128,-1.161536,-0.418427,-1.588743,0.015588,-1.518989
5,-0.858445,-0.942859,-0.489305,-0.12973,-0.18104,-1.80607,-0.967031,0.026938,0.494567,0.193614,...,-0.366102,0.036765,0.082684,0.025156,-1.180128,-1.161536,-0.418427,-1.588743,0.015588,-1.518989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280,-0.858445,-0.942859,-0.489305,-0.12973,-0.18104,-1.80607,-0.967031,0.026938,0.636627,0.193614,...,-0.366102,0.036765,0.082684,0.025156,-1.180128,-1.161536,-0.418427,-1.588743,0.015588,-1.518989
18281,-0.858445,-0.942859,-0.489305,-0.12973,-0.18104,-1.80607,-0.967031,0.026938,0.636627,0.193614,...,-0.366102,0.036765,0.082684,0.025156,-1.180128,-1.161536,-0.418427,-1.588743,0.015588,-1.518989
18282,-0.858445,-0.942859,-0.489305,-0.12973,-0.18104,-1.80607,-0.967031,0.026938,0.636627,0.193614,...,-0.366102,0.036765,0.082684,0.025156,-1.180128,-1.161536,-0.418427,-1.588743,0.015588,-1.518989


# 計算相似度

In [21]:
# 用cosine_similarity計算每個USER的相似度
cosine = cosine_similarity(final_item)
print(cosine.shape)
cosine

(5708, 5708)


array([[1.        , 0.98782338, 0.98770869, ..., 0.987053  , 0.97816604,
        0.98714561],
       [0.98782338, 1.        , 0.99988178, ..., 0.99943052, 0.99313719,
        0.99886576],
       [0.98770869, 0.99988178, 1.        , ..., 0.99931256, 0.993019  ,
        0.99874817],
       ...,
       [0.987053  , 0.99943052, 0.99931256, ..., 1.        , 0.99257244,
        0.99829686],
       [0.97816604, 0.99313719, 0.993019  , ..., 0.99257244, 1.        ,
        0.99265678],
       [0.98714561, 0.99886576, 0.99874817, ..., 0.99829686, 0.99265678,
        1.        ]])

In [22]:
#把對角線的值換成 0
np.fill_diagonal(cosine, 0 )
cosine

array([[0.        , 0.98782338, 0.98770869, ..., 0.987053  , 0.97816604,
        0.98714561],
       [0.98782338, 0.        , 0.99988178, ..., 0.99943052, 0.99313719,
        0.99886576],
       [0.98770869, 0.99988178, 0.        , ..., 0.99931256, 0.993019  ,
        0.99874817],
       ...,
       [0.987053  , 0.99943052, 0.99931256, ..., 0.        , 0.99257244,
        0.99829686],
       [0.97816604, 0.99313719, 0.993019  , ..., 0.99257244, 0.        ,
        0.99265678],
       [0.98714561, 0.99886576, 0.99874817, ..., 0.99829686, 0.99265678,
        0.        ]])

In [23]:
#把cosine array變成data frame, index是CustomerID
similarity_with_user = pd.DataFrame(cosine, index=final_item.index)

#把columns換成userId
similarity_with_user.columns = final_item.index
similarity_with_user

,0,1,2,3,4,5,6,7,8,9,...,5698,5699,5700,5701,5702,5703,5704,5705,5706,5707
CustomerID,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.987823,0.987709,0.987093,0.987916,0.987837,0.987517,0.987873,0.987831,0.987902,...,0.987821,0.987685,0.987709,0.987773,0.987054,0.987691,0.987764,0.987053,0.978166,0.987146
2,0.987823,0.000000,0.999882,0.999290,0.984584,0.999885,0.999786,0.999584,0.999588,0.999923,...,0.999998,0.999780,0.999919,0.999958,0.999639,0.999872,0.999939,0.999431,0.993137,0.998866
3,0.987709,0.999882,0.000000,0.999172,0.984468,0.999800,0.999668,0.999466,0.999470,0.999805,...,0.999880,0.999662,0.999801,0.999840,0.999521,0.999754,0.999820,0.999313,0.993019,0.998748
4,0.987093,0.999290,0.999172,0.000000,0.983889,0.999175,0.999076,0.998875,0.998879,0.999213,...,0.999288,0.999070,0.999210,0.999249,0.998929,0.999163,0.999229,0.998722,0.992427,0.998159
5,0.987916,0.984584,0.984468,0.983889,0.000000,0.984613,0.984237,0.984913,0.984813,0.984738,...,0.984586,0.984443,0.984468,0.984517,0.983929,0.984015,0.984509,0.983798,0.974805,0.984108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280,0.987691,0.999872,0.999754,0.999163,0.984015,0.999757,0.999659,0.999457,0.999461,0.999795,...,0.999870,0.999652,0.999792,0.999831,0.999515,0.000000,0.999812,0.999303,0.993012,0.998738
18281,0.987764,0.999939,0.999820,0.999229,0.984509,0.999823,0.999724,0.999522,0.999526,0.999861,...,0.999936,0.999718,0.999857,0.999897,0.999577,0.999812,0.000000,0.999369,0.993055,0.998804
18282,0.987053,0.999431,0.999313,0.998722,0.983798,0.999315,0.999217,0.999015,0.999019,0.999354,...,0.999428,0.999005,0.999350,0.999389,0.999070,0.999303,0.999369,0.000000,0.992572,0.998297


In [25]:
# 建立商品ID與商品名的 dataframe
items = df2.drop_duplicates(subset=['StockCode']).drop(columns=['CustomerID','quantity_Quartile','price_Quartile','buy_index_Quartile','rating']).sort_values(by=['StockCode']).reset_index(drop=True)
items.head()

,StockCode,ProductName
0,10002,桂冠大沙拉
1,10080,水手牌鬆餅預拌粉
2,10120,老奶奶檸檬蛋糕
3,10123C,收藏家電子防潮箱 CF-87
4,10124A,棉花共和國中空快乾浴巾-藍色


In [26]:
# 找出兩個使用者購買的相同商品
def get_user_similar_retailer( user1, user2 ):
    common_items = Rating_avg[Rating_avg.CustomerID == user1].merge(
    Rating_avg[Rating_avg.CustomerID == user2],
    on = "StockCode",
    how = "inner" )
    return common_items.merge( items, on = 'StockCode' )

#檢查是否合理
a = get_user_similar_retailer(1,1000)
a = a.loc[ : , ['rating_x_x','rating_x_y','ProductName']] # a.loc[取全部,['欄位1','欄位2','欄位3']] --> 取出欄位1,2,3的所有值
a.head()

,rating_x_x,rating_x_y,ProductName
0,1.6,1.1,木柄菜匙
1,3.0,3.0,美琪天然T3抗菌洗手乳
2,3.8,3.4,歐邁福麵包餅乾香蒜奶油口味-300g
3,3.5,3.5,OP生物分解抗菌立體密封袋 L
4,3.2,2.7,3M新防蹣水洗枕幼兒型(附枕套)


In [27]:
#order = np.argsort(df.values, axis=1)[:, :n] 
#按做排序列，並返回該值排序後的index
#每一列變Series做降冪排列，取出前n個, index換成Top1~Topn
def find_n_neighbours(df,n):
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df
# top 30 neighbours for each user
sim_user_30 = find_n_neighbours(similarity_with_user,30)
sim_user_30.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
CustomerID,,,,,,,,,,,,,,,,,,,,,
1,16274,17908,17968,17976,12433,16352,14142,12842,14511,16478,...,17603,14494,15458,16753,13483,15626,12686,14047,16017,13108
2,464,34,15,17443,599,621,18268,1079,18087,520,...,862,54,16257,17788,933,961,1014,13967,165,738
3,107,18068,770,464,34,15,17443,599,18268,621,...,15100,947,664,17408,862,54,16257,17788,933,961
4,464,15,34,17443,599,18268,621,1079,18087,520,...,17408,862,54,16257,17788,933,961,1014,13967,165
5,1,17968,13311,8,1117,12638,17908,16352,13532,14142,...,16274,12371,16017,13067,14047,14511,15679,14386,14494,13732


In [28]:
# 計算使用者對物品的喜好分數
def User_item_score(user,item):
    a = sim_user_30[sim_user_30.index==user].values #抓出top30表格，index==userId的列，把30個值變成一個array
    b = a.squeeze().tolist() #array降一維度，並變成list ex:[0.2,0.4,,,,,0.6]
    c = final_item.loc[:,item]
    d = c[c.index.isin(b)]
    f = d[d.notnull()]
    avg_user = Mean.loc[Mean['CustomerID'] == user,'rating'].values[0]
    index = f.index.values.squeeze().tolist()
    corr = similarity_with_user.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score

User_item_score(1,'10124A')

2.5722170746502844

In [29]:
#使用者買過的商品ID放在同一列
Rating_avg = Rating_avg.astype({"StockCode": str}) #itemId 的值變成str
Item_user = Rating_avg.groupby(by = 'CustomerID')['StockCode'].apply(lambda x:','.join(x))
Item_user 

CustomerID
1        22664,22294,21385,21107,22379,21787,22633,1701...
2                                              22716,20697
3                                                    22802
4                                              21372,21369
5        20668,22748,22352,22867,20696,85064,84755,2128...
                               ...                        
18280    22467,22499,22084,22611,22358,22180,22495,8248...
18281            22716,23008,23007,22028,22037,22467,23209
18282    22699,22818,21270,21108,23174,22423,22424,2110...
18283    21982,22139,23380,85150,20727,22735,22030,2202...
18287    84507C,22419,23445,21556,23274,22644,22865,226...
Name: StockCode, Length: 5708, dtype: object

In [30]:
# 輸入使用者跟推薦商品數量 --> 推薦List
def user_based_recommend(user,n):
    Items_bought_by_user = final.columns[final[final.index==user].notna().any()].tolist()
    a = sim_user_30[sim_user_30.index==user].values
    b = a.squeeze().tolist()
    d = Item_user[Item_user.index.isin(b)]
    l = ','.join(d.values)
    Items_bought_by_similar_users = l.split(',')
    Items_under_consideration = list(set(Items_bought_by_similar_users)-set(list(map(str, Items_bought_by_user))))
    #Items_under_consideration = list(map(int, Items_under_consideration))
    score = []
    for item in Items_under_consideration:
        c = final_item.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['CustomerID'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_user.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'StockCode':Items_under_consideration,'score':score})
    top_n_recommendation = data.sort_values(by='score',ascending=False).head(n)
    Item_Name = top_n_recommendation.merge(items, how='inner', on='StockCode')
    Item_Names = Item_Name.ProductName.values.tolist()
    return Item_Names

In [31]:
# 結果
user_based_recommend(1000,10)

['UCC炭燒濾掛式咖啡-8gx24',
 '維他露大蘋果蘇打-250mlx24',
 '皇家榖堡皇家台東米(圓ㄧ)3Kg',
 '五月花蓬厚柔三層抽取衛生紙-110PCx72',
 '家樂福冷凍黑豬五花火鍋片(每盒約250克)',
 '西莎野菜牛肉100g6入/組',
 '舒味思葡萄柚口味汽水-330mlx24',
 '家樂福超值抗菌濃縮洗衣精',
 'YS加強六星潔牙棒棒L桶裝',
 '南亞保鮮膜真省包 3支/組']